<a href="https://colab.research.google.com/github/SKD151105/Skin-Cancer-Classification/blob/main/model-inference_V2/vit_model_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ======================================================
# Step 1: Imports
# ======================================================
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import ViTForImageClassification, ViTFeatureExtractor, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from PIL import Image
import os

# ======================================================
# Step 2: Load and preprocess data
# ======================================================
data = pd.read_csv('/content/drive/MyDrive/mini_proj_data/HAM10000/hmnist_28_28_RGB.csv')
X = data.drop(columns=['label']).values
y = data['label'].values

In [2]:
X

array([[192, 153, 193, ..., 185, 154, 177],
       [ 25,  14,  30, ...,  25,  14,  27],
       [192, 138, 153, ..., 136, 104, 117],
       ...,
       [132, 118, 118, ..., 172, 151, 145],
       [160, 124, 146, ..., 185, 162, 172],
       [175, 142, 121, ..., 175, 139, 126]])

In [3]:
y

array([2, 2, 2, ..., 0, 0, 6])

In [4]:
# Reshape flattened pixels → 28x28x3
X = X.reshape(-1, 28, 28, 3).astype(np.uint8)

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# ======================================================
# Step 3: Prepare HuggingFace feature extractor
# ======================================================
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')

# ======================================================
# Step 4: Create Custom Dataset class
# ======================================================
class HMNISTDataset(Dataset):
    def __init__(self, images, labels, extractor):
        self.images = images
        self.labels = labels
        self.extractor = extractor

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img = Image.fromarray(self.images[idx])
        inputs = self.extractor(images=img, return_tensors="pt")
        inputs = {k: v.squeeze(0) for k, v in inputs.items()}
        inputs['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return inputs

train_dataset = HMNISTDataset(X_train, y_train, feature_extractor)
val_dataset = HMNISTDataset(X_val, y_val, feature_extractor)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/vit/feature_extraction_vit.py:30: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [5]:
# ======================================================
# Step 5: Load ViT model for classification
# ======================================================
num_classes = len(np.unique(y))
model = ViTForImageClassification.from_pretrained(
    'google/vit-base-patch16-224-in21k',
    num_labels=num_classes
)

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# ======================================================
# Step 7: Define metric computation
# ======================================================
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}

# ======================================================
# Step 8: Train the model
# ======================================================

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./vit_output',
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir='./vit_logs',
    load_best_model_at_end=True,
    report_to="none"  # 🚫 disables wandb / tensorboard logging cleanly
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()



Epoch,Training Loss,Validation Loss,Accuracy
1,0.854300,0.688963,0.777833
2,0.602300,0.565250,0.821268
3,0.482200,0.517132,0.836745
4,0.382600,0.483916,0.846231


Epoch,Training Loss,Validation Loss,Accuracy
1,0.854300,0.688963,0.777833
2,0.602300,0.565250,0.821268
3,0.482200,0.517132,0.836745
4,0.382600,0.483916,0.846231
5,0.303400,0.481811,0.843235


TrainOutput(global_step=2505, training_loss=0.5244577950346255, metrics={'train_runtime': 1519.8046, 'train_samples_per_second': 26.359, 'train_steps_per_second': 1.648, 'total_flos': 3.104468219559813e+18, 'train_loss': 0.5244577950346255, 'epoch': 5.0})

In [11]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [7]:
# ======================================================
# Step 9: Evaluate model
# ======================================================
preds = trainer.predict(val_dataset)
y_pred = np.argmax(preds.predictions, axis=1)
print("Validation Accuracy:", accuracy_score(y_val, y_pred))
print(classification_report(y_val, y_pred))

# ======================================================
# Step 10: Inference on all data & Save to CSV
# ======================================================
inference_dataset = HMNISTDataset(X, y, feature_extractor)
inference_loader = DataLoader(inference_dataset, batch_size=16)

model.eval()
results = []

with torch.no_grad():
    for i, batch in enumerate(inference_loader):
        inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        preds = probs.argmax(dim=-1).cpu().numpy()
        confs = probs.max(dim=-1).values.cpu().numpy()

        for j in range(len(preds)):
            results.append({
                'image_id': i * 16 + j,
                'true_label': int(batch['labels'][j]),
                'predicted_label': int(preds[j]),
                'confidence': float(confs[j])
            })

# Save results to CSV
results_df = pd.DataFrame(results)
results_df.to_csv('/content/drive/MyDrive/mini_proj_data/vit_inference_results.csv', index=False)
print("Inference results saved to vit_inference_results.csv ✅")

Validation Accuracy: 0.8432351472790813
              precision    recall  f1-score   support

           0       0.71      0.57      0.63        65
           1       0.69      0.77      0.73       103
           2       0.62      0.65      0.64       220
           3       0.00      0.00      0.00        23
           4       0.93      0.94      0.93      1341
           5       0.87      0.93      0.90        28
           6       0.66      0.65      0.66       223

    accuracy                           0.84      2003
   macro avg       0.64      0.64      0.64      2003
weighted avg       0.83      0.84      0.84      2003



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Inference results saved to vit_inference_results.csv ✅


In [8]:
!cp -r /content/vit_output /content/drive/MyDrive/mini_proj_data/
